# Installations and Imports

In [1]:
!pip install transformers datasets

In [2]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import pandas as pd
import os
import re
import csv
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import AutoTokenizer,create_optimizer,TFAutoModel

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
BATCH_SIZE=128
MAX_LENGTH=64

# Dataset Preparation

## Downloads

In [5]:
!pip install aicrowd-cli

In [6]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/3S1o6ABZeF3gdOc-dv8A8u_3-VZXQRQQJF5M9p91rOk
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/3S1o6ABZeF3gdOc-dv8A8u_3-VZXQRQQJF5M9p91rOk'
API Key valid
Gitlab access token valid
Saved details successfully!


In [7]:
!aicrowd dataset download -c esci-challenge-for-improving-product-search

test_public-v0.3.csv.zip:   0% 0.00/394k [00:00<?, ?B/s]
sample_submission_public-v0.3.csv.zip:   0% 0.00/331k [00:00<?, ?B/s]

test_public-v0.3.csv.zip: 100% 394k/394k [00:00<00:00, 12.5MB/s]
sample_submission_public-v0.3.csv.zip: 100% 331k/331k [00:00<00:00, 9.33MB/s]


train-v0.3.csv.zip: 100% 6.80M/6.80M [00:00<00:00, 45.5MB/s]

sample_submission_public-v0.3.csv.zip: 100% 812k/812k [00:00<00:00, 9.22MB/s]

product_catalogue-v0.3.csv.zip:   0% 0.00/657M [00:00<?, ?B/s]

test_public-v0.3.csv.zip:   0% 0.00/2.94M [00:00<?, ?B/s]


train-v0.3.csv.zip:   0% 0.00/19.8M [00:00<?, ?B/s]

test_public-v0.3.csv.zip: 100% 2.94M/2.94M [00:00<00:00, 17.7MB/s]


product_catalogue-v0.3.csv.zip:  10% 33.6M/328M [00:01<00:08, 33.5MB/s]


train-v0.3.csv.zip: 100% 19.8M/19.8M [00:00<00:00, 20.2MB/s]



sample_submission_public-v0.3.csv.zip: 100% 803k/803k [00:00<00:00, 18.0MB/s]

product_catalogue-v0.3.csv.zip:   5% 33.6M/657M [00:01<00:24, 25.3MB/s]


test_public-v0.3.csv.zip:   0% 0.00/2.94M [00:00<

## Decompress and Copy

In [8]:
!unzip "/content/train-v0.3.csv.zip" -d "/content/dataset/"

Archive:  /content/train-v0.3.csv.zip
  inflating: /content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv  


In [9]:
!unzip "/content/product_catalogue-v0.3.csv.zip" -d "/content/dataset/"

Archive:  /content/product_catalogue-v0.3.csv.zip
data/processed/public/task_2_multiclass_product_classification/product_catalogue-v0.3.csv:  mismatching "local" filename (data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv),
         continuing with "central" filename version
  inflating: /content/dataset/data/processed/public/task_2_multiclass_product_classification/product_catalogue-v0.3.csv  
  error:  invalid compressed data to inflate


In [15]:
!cp /content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv /content/drive/MyDrive/kdd_cup

In [16]:
!cp /content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv /content/drive/MyDrive/kdd_cup

cp: cannot stat '/content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv': No such file or directory


## Data Loading

In [22]:
filepath_train='/content/drive/MyDrive/kdd_cup/train-v0.3.csv'

In [23]:
filepath_catalogue='/content/drive/MyDrive/kdd_cup/product_catalogue-v0.3.csv'

In [24]:
df_catalogue=pd.read_csv(filepath_catalogue)

In [25]:
df_train=pd.read_csv(filepath_train)

In [26]:
df_train.loc[df_train['query_locale'] == 'jp']

,example_id,query,product_id,query_locale,substitute_label
1522347,1522347,-サングラス,B000MUWXKS,jp,no_substitute
1522348,1522348,-サングラス,B00M7TR75I,jp,no_substitute
1522349,1522349,-サングラス,B01GDMKFEY,jp,no_substitute
1522350,1522350,-サングラス,B01NCOWVAK,jp,no_substitute
1522351,1522351,-サングラス,B06WVHDWVR,jp,no_substitute
...,...,...,...,...,...
1834739,1834739,針なしほっちきす,B08XGQ9RH7,jp,substitute
1834740,1834740,針なしほっちきす,B0987RGRF2,jp,no_substitute
1834741,1834741,針なしほっちきす,B099NFJWP6,jp,no_substitute
1834742,1834742,針なしほっちきす,B09F3B413J,jp,no_substitute


In [27]:
df_catalogue

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),NaN,NaN,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,NaN,NaN,11 Degrees,NaN,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,NaN,NaN,11 Degrees,Negro (,es
...,...,...,...,...,...,...,...
1815211,B09FJYBVKS,スポーツブラ 揺れない ヨガウェア レディース トップス 通気性 ナイトブラ スポーツブラジ...,<p>【注意事項】</p><p>※商品内容以外のアクセサリー・小物等は付属しません.</p>...,シンプルのデザインで合わせやすいです。コート、半袖、シャツを合わせて、家で着てもいい、寝てい...,Doworspaw,ベージュ,jp
1815212,B09FTJ3S1N,ブラジャー レディース 補正ブラ Gabrioir 揺れない 大きいサイズ バストアップ美胸...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,パープル,jp
1815213,B09G72LDQZ,レディース ブラジャー 補正ブラ トップス Gabrioir 大きいサイズ 揺れない 脇高設...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,ベージュ,jp
1815214,B09GBGRTPB,ナイトブラ ノンワイヤーブラ ブラジャー レディース Gabrioir 大きいサイズ パット...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,パープル,jp


In [31]:
def get_product_title(product_id):
  options=[]
  options.append(str(product_id))
  matched_row=df_catalogue.loc[df_catalogue['product_id'].isin(options)]
  #print(int(str(matched_row['product_title']).split("   ")[0]))
  return matched_row['product_title'][int(str(matched_row['product_title']).split("   ")[0])]

In [29]:
get_product_title("B08N6NCKRX")### irrelevant

'面部保护 防雾 耐用 5个 透明面罩 健康盾牌 塑料面巾 完全可见 透气 可重复使用 成人 口罩 方便 打招呼 防护 (Clear)'

## Dataset Generation

In [33]:
header = ['query', 'product', 'label']

with open('/content/drive/MyDrive/kdd_cup/csv_file_9.csv', 'w', encoding='UTF8') as f:

  # create the csv writer
  writer = csv.writer(f)

  # write the header
  writer.writerow(header)

  for i in range(10):  # Assuming you want 10 files
    header = ['query', 'product', 'label']
    filepath = f'/content/drive/MyDrive/kdd_cup/csv_file_{i}.csv'
    with open(filepath, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)


        start_index = i * 2000
        end_index = start_index + 2000
        for index in range(start_index, end_index):
            # Replace this placeholder with your existing logic to extract 'query', 'product', and 'esci_label'
            query = df_train[index:index+1]['query'][index]
            product = get_product_title(df_train[index:index+1]['product_id'][index])
            esci_label = df_train[index:index+1]['substitute_label'][index]
            data = [query, product, esci_label]
            writer.writerow(data)


In [35]:
filepath="/content/drive/MyDrive/kdd_cup/csv_file_9.csv"
train_data=pd.read_csv(filepath)

In [36]:
train_data

,query,product,label
0,cajero automatico,"InnerSetting - Hucha para niños, automática, e...",substitute
1,cajero automatico,Hucha electrónica automática Cajero automático...,substitute
2,cajero automatico,PQZATX Hucha ElectróNica Caja De Dinero De Hue...,substitute
3,cajero automatico,Yzhongx Caja De Dinero De La Novedad Hucha Gra...,no_substitute
4,cajero automatico,Cajero automático Hucha Hucha Hucha Juguete Ca...,no_substitute
...,...,...,...
1995,camino de mesa,Delindo Lifestyle Samba Camino de mesa antiman...,no_substitute
1996,camino de mesa,Camino de Mesa Geometría Gris Mantel de Lino d...,no_substitute
1997,camino de mesa,Camino de mesa moderno de Matedepreso - Durade...,no_substitute
1998,camino de mesa menta,Top Finel Rayas de Color Corredor de la Tabla ...,substitute


## Data Preprocessing

In [39]:
filepaths=["/content/drive/MyDrive/kdd_cup/csv_file_"+str(i)+".csv" for i in range(10)]
print(filepaths)

['/content/drive/MyDrive/kdd_cup/csv_file_0.csv', '/content/drive/MyDrive/kdd_cup/csv_file_1.csv', '/content/drive/MyDrive/kdd_cup/csv_file_2.csv', '/content/drive/MyDrive/kdd_cup/csv_file_3.csv', '/content/drive/MyDrive/kdd_cup/csv_file_4.csv', '/content/drive/MyDrive/kdd_cup/csv_file_5.csv', '/content/drive/MyDrive/kdd_cup/csv_file_6.csv', '/content/drive/MyDrive/kdd_cup/csv_file_7.csv', '/content/drive/MyDrive/kdd_cup/csv_file_8.csv', '/content/drive/MyDrive/kdd_cup/csv_file_9.csv']


In [40]:
dataset = load_dataset('csv', data_files=filepaths)

Generating train split: 0 examples [00:00, ? examples/s]

In [41]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'product', 'label'],
        num_rows: 20000
    })
})

In [42]:
dataset['train'][0]

{'query': '11 degrees',
 'product': '11 Degrees de los Hombres Playera con Logo, Negro, L',
 'label': 'no_substitute'}

In [43]:
def get_label(label):
  if label=='exact':
    return 1.0
  elif label=='substitute':
    return 0.7
  elif label=='complement':
    return 0.5
  else:
    return 0.0

In [44]:
model_id="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [45]:
def preprocess(dataset):
  if dataset['product']==None:
    dataset['product']=dataset['query']

  dataset['input_ids_query']=[]
  dataset['token_type_ids_query']=[]
  dataset['attention_mask_query']=[]

  dataset['input_ids_product']=[]
  dataset['token_type_ids_product']=[]
  dataset['attention_mask_product']=[]

  tokenized_output_query=tokenizer(dataset['query'],max_length=MAX_LENGTH,padding='max_length',truncation=True)
  tokenized_output_product=tokenizer(dataset['product'],max_length=MAX_LENGTH,padding='max_length',truncation=True)

  dataset['input_ids_query'].append(tokenized_output_query['input_ids'])
  dataset['token_type_ids_query'].append(tokenized_output_query['token_type_ids'])
  dataset['attention_mask_query'].append(tokenized_output_query['attention_mask'])

  dataset['input_ids_product'].append(tokenized_output_product['input_ids'])
  dataset['token_type_ids_product'].append(tokenized_output_product['token_type_ids'])
  dataset['attention_mask_product'].append(tokenized_output_product['attention_mask'])

  dataset['label']=get_label(dataset['label'])

  return dataset

In [46]:
prep_dataset=dataset.map(preprocess)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [47]:
prep_dataset['train'][21:22]

{'query': ['144 preservativos'],
 'product': ['Durex Preservativos Dame Placer con Puntos y Estrías - Total 12 Condones'],
 'label': [0.7],
 'input_ids_query': [[[0,
    51332,
    122785,
    26465,
    2,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1]]],
 'token_type_ids_query': [[[0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
   

In [48]:
tf_dataset = prep_dataset["train"].to_tf_dataset(
    columns=['input_ids_query', 'token_type_ids_query', 'attention_mask_query','input_ids_product', 'token_type_ids_product', 'attention_mask_product', 'label'],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

In [49]:
tf_dataset

<_PrefetchDataset element_spec={'label': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'input_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'input_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None)}>

In [50]:
for i in tf_dataset.take(1):
  print(i)

{'label': <tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0.7, 0. , 0. , 0. , 0. ,
       0.7, 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0.7, 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0.7, 0. , 0. , 0. , 0. , 0.7, 0. , 0.7, 0. , 0.7, 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0.7, 0.7, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0.7, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0.7,
       0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0. , 0. , 0.7, 0. ],
      dtype=float32)>, 'input_ids_query': <tf.Tensor: shape=(128, 1, 64), dtype=int64, numpy=
array([[[     0,    337,   2311, ...,      1,      1,      1]],

       [[     0,     10,    748, ...,      1,      1,      1]],

       

# Modeling

In [51]:
model = TFAutoModel.from_pretrained(model_id)
model.summary()

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  117653760 
                                                                 
Total params: 117653760 (448.81 MB)
Trainable params: 117653760 (448.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
class SentenceTransformer(tf.keras.Model):
  def __init__(self,model):
    super(SentenceTransformer,self).__init__()
    self.model=model
    self.dense=Dense(1,activation='sigmoid')

  def compile(self,optimizer,loss_fn):
    super(SentenceTransformer,self).compile()
    self.optimizer=optimizer
    self.loss_fn=loss_fn
    self.loss_metric=tf.keras.metrics.Mean(name='loss')

  @property
  def metrics(self):
    return [self.loss_metric]

  def mean_pooling(self, model_output, attention_mask):
    token_embeddings = model_output[0]

    input_mask_expanded = tf.cast(
        tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
        tf.float32
    )
    return tf.math.reduce_sum(token_embeddings * input_mask_expanded, axis=1)/tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, tf.float32.max)

  def train_step(self,train_data):

    query={'input_ids':train_data['input_ids_query'][:,0,:],
           'token_type_ids':train_data['token_type_ids_query'][:,0,:],
           'attention_mask':train_data['attention_mask_query'][:,0,:]}

    product={'input_ids':train_data['input_ids_product'][:,0,:],
             'token_type_ids':train_data['token_type_ids_product'][:,0,:],
             'attention_mask':train_data['attention_mask_product'][:,0,:]}

    labels=train_data['label']

    with tf.GradientTape() as recorder:
      query_predictions=self.model(query)
      pred_query=self.mean_pooling(query_predictions,train_data['attention_mask_query'][:,0,:])

      product_predictions=self.model(product)
      pred_product=self.mean_pooling(product_predictions,train_data['attention_mask_product'][:,0,:])

      pred_concat=tf.concat([pred_query,pred_product,tf.abs(pred_query-pred_product)],axis=-1)

      predictions=self.dense(pred_concat)
      loss=self.loss_fn(labels,predictions)


    partial_derivatives = recorder.gradient(loss,self.model.trainable_weights)
    self.optimizer.apply_gradients(zip(partial_derivatives, self.model.trainable_weights))


    self.loss_metric.update_state(loss)

    return {'loss':self.loss_metric.result(),}

# Model Training

In [53]:
stransformer=SentenceTransformer(model)
stransformer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5,),
    loss_fn=tf.keras.losses.BinaryCrossentropy(),
)

In [54]:
EPOCHS=5
history=stransformer.fit(tf_dataset,epochs=EPOCHS,)

Epoch 1/5


157/157 [==============================] - 232s 982ms/step - loss: 0.3253
Epoch 2/5
157/157 [==============================] - 156s 990ms/step - loss: 0.2547
Epoch 3/5
157/157 [==============================] - 155s 989ms/step - loss: 0.2222
Epoch 4/5
157/157 [==============================] - 155s 989ms/step - loss: 0.1874
Epoch 5/5
157/157 [==============================] - 155s 989ms/step - loss: 0.1537


In [64]:
model_path='/content/drive/MyDrive/stransformer/stransformers.h5'

In [62]:
#stransformer.model.save_weights(model_path)

In [65]:
model.load_weights(model_path)

In [66]:
def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]

  input_mask_expanded = tf.cast(
      tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
      tf.float32
  )
  return tf.math.reduce_sum(token_embeddings * input_mask_expanded, axis=1)/tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, tf.float32.max)

# Testing

## Create Embeddings

In [67]:
filepath_catalogue='/content/drive/MyDrive/kdd_cup/product_catalogue-v0.3.csv'
df_catalogue=pd.read_csv(filepath_catalogue)

In [68]:
df_catalogue['product_title'][1000000]

'【国内正規品】PeakDesign ピークデザイン シェルS SH-S-1'

In [69]:
product_titles=[str(df_catalogue['product_title'][i]) for i in range(len(df_catalogue))]
#print(product_titles)

In [70]:
print(len(product_titles))

1815216


In [73]:
embeddings=[]

In [74]:
INFERENCE_BATCH_SIZE=640
len(product_titles)//INFERENCE_BATCH_SIZE

2836

In [75]:
for i in range(len(product_titles)//INFERENCE_BATCH_SIZE):
  tokenized_output=tokenizer(
      product_titles[INFERENCE_BATCH_SIZE*i:INFERENCE_BATCH_SIZE*(i+1)],max_length=MAX_LENGTH,padding='max_length',truncation=True,return_tensors="tf")
  model_output=model(tokenized_output)
  embedding=mean_pooling(model_output,tokenized_output['attention_mask'])
  embeddings.append(embedding)
  if i%100==0:
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [76]:
embeddings

[<tf.Tensor: shape=(640, 384), dtype=float32, numpy=
 array([[ 0.3773438 ,  0.35455045, -0.32074016, ..., -0.5497283 ,
         -0.3747677 , -0.15803736],
        [ 0.09945303,  0.06671882,  0.5101331 , ..., -0.09471593,
         -0.07403563, -0.07611595],
        [ 0.17933683,  0.34300077, -0.28494066, ..., -0.46823773,
         -0.20887184,  0.33857334],
        ...,
        [-0.02904113,  0.08939078,  0.2953545 , ...,  0.19294824,
         -0.4888678 ,  0.23521686],
        [ 0.13134119, -0.4129755 ,  0.671449  , ..., -0.08000043,
         -0.374243  ,  0.14118996],
        [ 0.12613553, -0.25654092,  0.22295348, ..., -0.7033478 ,
         -0.26241776, -0.08694402]], dtype=float32)>,
 <tf.Tensor: shape=(640, 384), dtype=float32, numpy=
 array([[-0.24924405, -0.17198543,  0.47206503, ..., -0.25561258,
         -0.23905914,  0.26072365],
        [-0.13801716, -0.08104306,  0.20557031, ...,  0.0605973 ,
          0.07992052, -0.03561874],
        [ 0.08461614,  0.11214278, -0.1583648 ,

In [78]:
np.savez_compressed('embeddings.npz', embeddings)
np.savez_compressed('product_titles.npz',product_titles)

In [79]:
!cp '/content/embeddings.npz' '/content/drive/MyDrive/stransformer/'
!cp '/content/product_titles.npz' '/content/drive/MyDrive/stransformer/'

## Use Embeddings

In [80]:
loaded_embedding=np.load('/content/drive/MyDrive/stransformer/embeddings.npz')
loaded_embedding_array=np.array(loaded_embedding['arr_0'])

In [81]:
loaded_titles=np.load('/content/drive/MyDrive/stransformer/product_titles.npz')
loaded_titles_array=np.array(loaded_titles['arr_0'])

In [82]:
loaded_embedding_array.shape

(2836, 640, 384)

In [83]:
loaded_embedding_array=loaded_embedding_array.reshape(-1,loaded_embedding_array.shape[2])
print(loaded_embedding_array.shape)

(1815040, 384)


In [84]:

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [85]:
inputs = tokenizer(["針なしほっちきす"],max_length=MAX_LENGTH,padding='max_length',truncation=True,return_tensors="tf")

logits = model(**inputs)
out_embedding=mean_pooling(logits,inputs['attention_mask'])
print(out_embedding.shape)

(1, 384)


In [86]:
u_dot_v=np.matmul(loaded_embedding_array,(np.array(out_embedding).T))
print(u_dot_v.shape)

(1815040, 1)


In [87]:
u_magnitude=np.sqrt(np.sum(loaded_embedding_array*loaded_embedding_array,axis=-1))
print(u_magnitude.shape)
print(u_magnitude)

(1815040,)
[6.9042087 6.9365306 6.083935  ... 7.077644  7.6089096 6.173305 ]


In [88]:
v_magnitude=np.sqrt(np.sum(out_embedding*out_embedding,axis=-1))
print(v_magnitude.shape)
print(v_magnitude)

(1,)
[3.8864603]


In [89]:
cosine_similarity=u_dot_v.T/(u_magnitude*v_magnitude)
print(cosine_similarity)

[[0.14799151 0.15982823 0.2981542  ... 0.21259536 0.28617004 0.02770919]]


In [90]:
sorted_indices=np.argsort(cosine_similarity,axis=-1)
print(sorted_indices)

[[ 556043  554263  524846 ... 1147245  985869 1810217]]


In [91]:
for i in range(25):
  print(i,loaded_titles_array[sorted_indices[:,len(sorted_indices[0])-i-1]])

0 ['むかしばなし2']
1 ['読んで効くタウリンのはなし']
2 ['割れないシャボン玉']
3 ['割れないシャボン玉']
4 ['さまよう刃']
5 ['さまよう刃']
6 ['お化けなんてないさ']
7 ['お化けなんてないさ']
8 ['おうまでこばなし']
9 ['つみきのいえ']
10 ['耳なし芳一']
11 ['耳なし芳一']
12 ['耳なし芳一']
13 ['タイトルなし']
14 ['字のないはがき']
15 ['フンころがさず']
16 ['もりのとしょかん']
17 ['ねこむかしばなし']
18 ['そのぬくもりに用がある']
19 ['鉄火幾松ぶし']
20 ['くまのがっこう だいすきおとつみき']
21 ['おもらしまんが']
22 ['おらおらでひとりいぐも (河出文庫)']
23 ['あむんぜん']
24 ['ひとつむぎの手']


In [92]:
small_dataset = dataset.filter(lambda example: example["query"].startswith("A"))
print(len(small_dataset))

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

1
